# Furniture Product Data Analytics

This notebook performs exploratory data analysis (EDA) on the furniture product dataset.
We will analyze product brands, pricing distributions, and other key characteristics to understand
the data before building our recommendation system.

## Step 1: Import Required Libraries

We import pandas for data manipulation and plotly for interactive visualizations.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

## Step 2: Load the Dataset

Loading the furniture dataset from Google Drive. This dataset contains product information including
titles, descriptions, prices, brands, images, and other metadata.

In [ ]:
# Load dataset from Google Drive URL
url = "https://drive.google.com/uc?export=download&id=1uD1UMXT2-13GQkbH9NmEOyUVI-zKyl6"

try:
    df = pd.read_csv(url)
    print(f"✓ Dataset loaded successfully")
    print(f"✓ Total records: {len(df):,}")
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please check the URL or your internet connection")

## Step 3: Initial Data Exploration

Let's examine the structure and content of our dataset to understand what we're working with.

In [ ]:
# Display first few rows
print("=" * 80)
print("DATASET PREVIEW (First 5 rows)")
print("=" * 80)
display(df.head())

In [ ]:
# Display dataset information
print("\n" + "=" * 80)
print("DATASET INFORMATION")
print("=" * 80)
print(f"\nShape: {df.shape[0]} rows × {df.shape[1]} columns\n")
df.info()

In [ ]:
# Check for missing values
print("\n" + "=" * 80)
print("MISSING VALUES ANALYSIS")
print("=" * 80)

missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing Count': df.isnull().sum().values,
    'Missing %': (df.isnull().sum().values / len(df) * 100).round(2)
})

missing_data = missing_data[missing_data['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_data) > 0:
    display(missing_data)
else:
    print("✓ No missing values found in the dataset!")

## Step 4: Statistical Summary

Understanding the distribution and basic statistics of numerical columns, particularly pricing.

In [ ]:
# Statistical summary for numerical columns
print("=" * 80)
print("STATISTICAL SUMMARY")
print("=" * 80)
display(df.describe())

## Step 5: Visualization 1 - Top 10 Product Brands

This bar chart shows the distribution of products across the top 10 brands.
Understanding brand distribution helps us identify popular manufacturers and potential
biases in our recommendation system.

In [ ]:
# Calculate top 10 brands by product count
top_brands = df['brand'].value_counts().head(10).reset_index()
top_brands.columns = ['Brand', 'Product Count']

# Create interactive bar chart
fig = px.bar(
    top_brands,
    x='Brand',
    y='Product Count',
    title='Top 10 Furniture Brands by Product Count',
    labels={'Brand': 'Brand Name', 'Product Count': 'Number of Products'},
    color='Product Count',
    color_continuous_scale='Teal',
    text='Product Count'
)

# Customize layout
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title='Brand',
    yaxis_title='Number of Products',
    showlegend=False,
    height=500,
    font=dict(size=12),
    hovermode='x unified'
)

fig.show()

print(f"\n✓ Top brand: {top_brands.iloc[0]['Brand']} with {top_brands.iloc[0]['Product Count']:,} products")

## Step 6: Visualization 2 - Price Distribution Analysis

This histogram shows the distribution of product prices, excluding extreme outliers above $3000.
Understanding price distribution is crucial for:
- Setting appropriate price ranges in recommendations
- Identifying pricing patterns and market segments
- Detecting outliers and anomalies in the data

In [ ]:
# Filter out extreme outliers (prices above $3000)
price_threshold = 3000
df_filtered = df[df['price'] <= price_threshold].copy()

print(f"Original dataset: {len(df):,} products")
print(f"After filtering (price ≤ ${price_threshold}): {len(df_filtered):,} products")
print(f"Outliers removed: {len(df) - len(df_filtered):,} products ({((len(df) - len(df_filtered)) / len(df) * 100):.2f}%)")

# Create histogram
fig = px.histogram(
    df_filtered,
    x='price',
    nbins=50,
    title=f'Product Price Distribution (Prices ≤ ${price_threshold})',
    labels={'price': 'Price (USD)', 'count': 'Number of Products'},
    color_discrete_sequence=['#1a8b8b']
)

# Add mean and median lines
mean_price = df_filtered['price'].mean()
median_price = df_filtered['price'].median()

fig.add_vline(
    x=mean_price,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean: ${mean_price:.2f}",
    annotation_position="top"
)

fig.add_vline(
    x=median_price,
    line_dash="dash",
    line_color="blue",
    annotation_text=f"Median: ${median_price:.2f}",
    annotation_position="top right"
)

# Customize layout
fig.update_layout(
    xaxis_title='Price (USD)',
    yaxis_title='Number of Products',
    height=500,
    showlegend=False,
    hovermode='x unified'
)

fig.show()

# Display price statistics
print("\n" + "=" * 80)
print("PRICE STATISTICS (Filtered Data)")
print("=" * 80)
print(f"Mean Price: ${mean_price:.2f}")
print(f"Median Price: ${median_price:.2f}")
print(f"Min Price: ${df_filtered['price'].min():.2f}")
print(f"Max Price: ${df_filtered['price'].max():.2f}")
print(f"Standard Deviation: ${df_filtered['price'].std():.2f}")

## Step 7: Additional Insights - Category Analysis

Understanding product categories helps inform our recommendation engine.

In [ ]:
# Analyze categories if column exists
if 'categories' in df.columns:
    print("=" * 80)
    print("CATEGORY ANALYSIS")
    print("=" * 80)
    
    # Count unique categories
    print(f"\nTotal unique categories: {df['categories'].nunique():,}")
    
    # Top 10 categories
    top_categories = df['categories'].value_counts().head(10)
    print("\nTop 10 Categories:")
    for idx, (category, count) in enumerate(top_categories.items(), 1):
        print(f"{idx}. {category}: {count:,} products")
else:
    print("Categories column not found in dataset")

## Step 8: Data Quality Summary

Final summary of data quality and readiness for the ML pipeline.

In [ ]:
print("=" * 80)
print("DATA QUALITY SUMMARY")
print("=" * 80)
print(f"\n✓ Total Products: {len(df):,}")
print(f"✓ Total Columns: {len(df.columns)}")
print(f"✓ Missing Values: {df.isnull().sum().sum():,} ({(df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100):.2f}% of total cells)")

# Check key columns for ML pipeline
key_columns = ['title', 'description', 'brand', 'price', 'images', 'uniq_id']
print("\nKey Columns for ML Pipeline:")
for col in key_columns:
    if col in df.columns:
        missing = df[col].isnull().sum()
        status = "✓" if missing == 0 else f"⚠ {missing} missing"
        print(f"  {status} {col}")
    else:
        print(f"  ✗ {col} - NOT FOUND")

print("\n" + "=" * 80)
print("✓ Data analytics complete! Ready to proceed with model training.")
print("=" * 80)